In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

import util as u

from glob import glob
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 2000)


In [4]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("test/*/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("test/*/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

#is_file = os.path.isfile("match_up.txt")

#if is_file:
#    with open("match_up.txt", "r") as tf:
#        match_up_list = tf.read().split(',')[:-1]
#    #途中から始めるためのコード書く
#    target_txts_list = set(result_file) - set(match_up_list)
#else:
#    target_txts_list = result_file

target_info_list = main_info_file[0:1]
target_result_list = main_result_file[0:1]

In [5]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

#New_col = 'insert_1_1 insert_1_2 insert_2_1 insert_2_2 insert_3_1 insert_3_2 insert_4_1 insert_4_2 insert_5_1 insert_5_2 insert_6_1 insert_6_2 insert_7_1 insert_7_2 \
#            start_timing_1_1 start_timing_1_2 start_timing_2_1 start_timing_2_2 start_timing_3_1 start_timing_3_2 start_timing_4_1 start_timing_4_2 start_timing_5_1 start_timing_5_2 start_timing_6_1 start_timing_6_2 start_timing_7_1 start_timing_7_2 \
#            rank_1_1 rank_1_2 rank_2_1 rank_2_2 rank_3_1 rank_3_2 rank_4_1 rank_4_2 rank_5_1 rank_5_2 rank_6_1 rank_6_2 rank_7_1 rank_7_2'.split()

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [6]:
###後で消す↓###
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
###↑後で消す###

# info_htmlの加工をする
# race_id、枠、選手ID、級別、体重(直前情報から取る)、調整重量、F数、L数、平均ST、モーター2連、モーター3連、初日進入、
# 初日ST、初日成績、～ 、最終日、展示タイム、チルト、プロペラ、部品交換

for target_info in tqdm(target_info_list):
    for i in range(1, 7):
        target_info_day = target_info[-19:-5]
        Pit_No = f"{i:02}"
        
        #レース結果を保存したhtmlファイルとそのDF
        target_info_day = target_info[-19:-5]
        dfs = pd.read_html(target_info, encoding="utf-8")
        race_info1_df = dfs[0]
        
        #dfs[0]の整理
        df_player_info = u.split_inner_data(u.parse(race_info1_df), target_info_day)
        df_player_info = df_player_info.iloc[i-1]

        player_ID = df_player_info["pid"]

        #複勝率、三連率を計算
        player_win_rate = u.player_win_rate_cal(target_info_day, player_ID)
        se_player_win_rate = player_win_rate.iloc[0]

        #開催期間の艇番を表示
        df_bort_color = u.bort_color(target_info).iloc[i-1]
        #開催期間の艇番を表示
        df_open_grades = u.Open_Grades_sort(target_info).iloc[i-1]

        #天気情報を追加
        main_info_dummy = pd.concat([df_player_info, u.weather_condition(target_info)])
        #勝率を追加
        main_info_dummy = pd.concat([main_info_dummy, se_player_win_rate])
        #開催期間の艇番を追加
        main_info_dummy = pd.concat([main_info_dummy, df_bort_color])
        #開催中のレース成績
        main_info_dummy = pd.concat([main_info_dummy, df_open_grades])
        #ダミーの情報をメイン情報に追加
        main_info = main_info.append(main_info_dummy, ignore_index=True)


C:\Users\rstta\AppData\Local\Temp/ipykernel_11032/2234562932.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\rstta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [7]:
display(main_info.head(6))

,rid,en,pid,class,branch,weight,F,L,st,mnww,mnwww,idx,temp,whether,win_s,win_a,temp_w,wave_h,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3,bort_color_1_1,bort_color_1_2,bort_color_2_1,bort_color_2_2,bort_color_3_1,bort_color_3_2,bort_color_4_1,bort_color_4_2,bort_color_5_1,bort_color_5_2,bort_color_6_1,bort_color_6_2,bort_color_7_1,bort_color_7_2,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,01_20140102_01,1.0,3115,B1,群馬,50.0,0,0,0.22,0.00,-,01_20140102_01_01,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01_20140102_01,2.0,3560,B1,群馬,49.0,0,0,0.19,0.00,-,01_20140102_01_02,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01_20140102_01,3.0,3376,B1,群馬,48.5,0,0,0.24,14.20,-,01_20140102_01_03,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_20140102_01,4.0,3532,A1,群馬,52.8,0,0,0.17,0.00,-,01_20140102_01_04,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01_20140102_01,5.0,2808,B2,群馬,51.4,0,0,0.29,0.00,-,01_20140102_01_05,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01_20140102_01,6.0,4645,B1,群馬,50.4,0,0,0.16,66.60,-,01_20140102_01_06,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dfs = pd.read_html(target_info, encoding="utf-8")

display(dfs[1])

枠  写真 ボートレーサー      体重 展示タイム  チルト    プロペラ 部品交換 前走成績       
    枠  写真 ボートレーサー    調整重量 展示タイム  チルト    プロペラ 部品交換 前走成績 前走成績.1
0   1 NaN   鈴木 正人  50.0kg  6.85 -0.5  ヤ→ヤ(新)  NaN    R    NaN
1   1 NaN   鈴木 正人  50.0kg  6.85 -0.5  ヤ→ヤ(新)  NaN   進入    NaN
2   1 NaN   鈴木 正人     0.0  6.85 -0.5  ヤ→ヤ(新)  NaN   ST    NaN
3   1 NaN   鈴木 正人     0.0  6.85 -0.5  ヤ→ヤ(新)  NaN   着順    NaN
4   2 NaN   太田 克哉  49.0kg  6.85 -0.5       ヤ  NaN    R    NaN
5   2 NaN   太田 克哉  49.0kg  6.85 -0.5       ヤ  NaN   進入    NaN
6   2 NaN   太田 克哉     1.0  6.85 -0.5       ヤ  NaN   ST    NaN
7   2 NaN   太田 克哉     1.0  6.85 -0.5       ヤ  NaN   着順    NaN
8   3 NaN   萩原 善行  48.5kg  6.87 -0.5       ヤ  NaN    R    NaN
9   3 NaN   萩原 善行  48.5kg  6.87 -0.5       ヤ  NaN   進入    NaN
10  3 NaN   萩原 善行     1.5  6.87 -0.5       ヤ  NaN   ST    NaN
11  3 NaN   萩原 善行     1.5  6.87 -0.5       ヤ  NaN   着順    NaN
12  4 NaN    柴田 光  52.8kg  6.90 -0.5  ヤ→ヤ(新)  NaN    R    NaN
13  4 NaN    柴田 光  52.8kg  6.90 -0.5  ヤ→ヤ(新)  NaN   進入    NaN
14  4 NaN    柴田 光     0.0  6.90 -0.5  ヤ→ヤ(新)  NaN   ST    NaN
15  4 NaN    柴田 光     0.0  6.90 -0.5  ヤ→ヤ(新)  NaN   着順    NaN
16  5 NaN    田嶋 登  51.4kg  6.97 -0.5       ヤ  NaN    R    NaN
17  5 NaN    田嶋 登  51.4kg  6.97 -0.5       ヤ  NaN   進入    NaN
18  5 NaN    田嶋 登     0.0  6.97 -0.5       ヤ  NaN   ST    NaN
19  5 NaN    田嶋 登     0.0  6.97 -0.5       ヤ  NaN   着順    NaN
20  6 NaN   上村 純一  50.4kg  6.85 -0.5       ヤ  NaN    R    NaN
21  6 NaN   上村 純一  50.4kg  6.85 -0.5       ヤ  NaN   進入    NaN
22  6 NaN   上村 純一     0.0  6.85 -0.5       ヤ  NaN   ST    NaN
23  6 NaN   上村 純一     0.0  6.85 -0.5       ヤ  NaN   着順    NaN

In [9]:
target_info_list = main_info_file

In [35]:
dfs = pd.read_html("test/01/info_html/01_20140103_01.html", encoding="utf-8")[1]
dfs["rid"] = target_info[-19:-5]
dfs = dfs.drop("前走成績", axis=1)
display(dfs.loc[2, ("体重", "調整重量")])
display(dfs[::2][1::2])


C:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


'0.0'

,枠,写真,ボートレーサー,体重,展示タイム,チルト,プロペラ,部品交換,rid
,枠,写真,ボートレーサー,調整重量,展示タイム,チルト,プロペラ,部品交換,
2,1,NaN,森下 勇,0.0,6.66,-0.5,ヤ,NaN,01_20150514_11
6,2,NaN,永瀬 巧,0.0,6.69,-0.5,ヤ,NaN,01_20150514_11
10,3,NaN,吉村 茂樹,0.0,6.66,-0.5,ヤ,NaN,01_20150514_11
14,4,NaN,山崎 智也,0.0,6.70,-0.5,ヤ,NaN,01_20150514_11
18,5,NaN,金澤 一洋,0.0,6.68,-0.5,ヤ,ピストン×２ シャフト,01_20150514_11
22,6,NaN,椎名 豊,0.0,6.71,-0.5,ヤ,NaN,01_20150514_11


In [102]:
parts_col = "Adj_weight Open_t tilt pera pis shaft gia cab electro ring silinda calbo".split()
#ギヤ ピストン シャフト キャブ 電気 リング シリンダ キャリアボディー

df_parts_info = pd.DataFrame(index=[], columns=parts_col)

dfs = pd.read_html(target_info, encoding="utf-8")[1]
dfs["rid"] = target_info[-19:-5]
dfs = dfs.drop("前走成績", axis=1)

df = dfs[::2][1::2]
df.reset_index(drop=True, inplace=True) 
display(df)

display(df.loc[0, "展示タイム"])

C:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,枠,写真,ボートレーサー,体重,展示タイム,チルト,プロペラ,部品交換,rid
,枠,写真,ボートレーサー,調整重量,展示タイム,チルト,プロペラ,部品交換,
0,1,NaN,峰 竜太,0.0,6.65,-0.5,ヤ,NaN,01_20150514_11
1,2,NaN,伯母 芳恒,0.0,6.68,-0.5,ヤ,NaN,01_20150514_11
2,3,NaN,今井 貴士,0.0,6.73,-0.5,ヤ,NaN,01_20150514_11
3,4,NaN,鶴本 崇文,0.0,6.74,-0.5,ヤ,NaN,01_20150514_11
4,5,NaN,亀山 高雅,0.0,6.75,-0.5,ヤ,NaN,01_20150514_11
5,6,NaN,永田 啓二,0.0,6.76,-0.5,ヤ,NaN,01_20150514_11


展示タイム    6.65
Name: 0, dtype: object

In [105]:
df = pd.read_csv("test.csv", encoding="utf-8", header=None)
df = df[~df[7].isnull()]

pattern_dict = {
    "ピストン×１":"null",
    "ピストン×２ シャフト":"null",
    "ギヤ":"null",
    "ピストン×２":"null",
    "キャブ":"null",
    "リング×１":"null",
    "電気":"null",
    "シャフト":"null"
}
output = [x for x in df[7] if pattern_dict.get(x)!="null"]
print (output)

['シャフト', 'シャフト', 'シャフト', 'シャフト']


### データが取れてないところからデータを取る処理を書く

### 現在開催されているデータを引っ張ってくる

### PyCaretってのを使ってみるよ